In [90]:
import time
from selenium import webdriver as wd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re

In [79]:
def scrape_comments(url):
    data1=[]
    data2=[]
    # Warning: need to change the location of the path where you saved the chrome driver
    driver = Chrome(executable_path=r'C:\Program Files\chromedriver.exe')  
    wait = WebDriverWait(driver,5)
    driver.get(url)
    driver.maximize_window()
    time.sleep(3)
    
    # Pause the video to avoid "autoplay next video"
    videoplayer = driver.find_element_by_id('movie_player')
    videoplayer.send_keys(Keys.SPACE) #hits space
    time.sleep(3)
    
    # Save variable names
    comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    vidTitle = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text

    # Scroll to view the comment section
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    # Scroll all the way down to the bottom
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load"
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far
        time.sleep(3)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")


    # Scrape each author name
    for body in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#author-text"))):
        data1.append(body.text)

    # Scrape each comment
    for body in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
        data2.append(body.text)

    driver.close()

    # Merge each author name with corresponding comment
    mergedData = ['&&&&&&'.join(x) for x in zip(data1,data2)]
    return mergedData, vidTitle
            
# html key for the Id and the comment:
# main
# author-text
# comment

In [80]:
def transform(scrapedData, videoTitle):
    # Convert a list to a dataframe
    df = pd.DataFrame(scrapedData, columns=['chunk'])

    # Add column names
    df[['User ID','Comment']] = df['chunk'].str.split("&&&&&&",expand=True,)

    # Drop unnecessary column(s)
    df = df.drop(columns=['chunk'])
    
    # Save the data as a csv file
    df.to_csv(videoTitle + '.csv', index=False)
    print("Showing the first five comments as an example")
    print("The order of the comments shown here may not be the same as what you see in the Youtube comment section")
    print('Go to where you saved this .ipynb file and you will see a new csv file named as the title of the Youtube video')
    return df.head()

In [76]:
url = "https://www.youtube.com/watch?v=bApcndK4kWw&ab_channel=BrianPwildlifeBrianPwildlife"

In [72]:
scrapedData, videoTitle = scrape_comments(url)

In [73]:
df = pd.DataFrame(scrapedData, columns=['chunk'])
df[['User ID','Comment']] = df['chunk'].str.split("&&&&&&",expand=True,)
df = df.drop(columns=['chunk'])


In [74]:
videoTitle

'Baby Penguin Chases Mom for More Food'

In [75]:
df.to_csv(videoTitle + '.csv', index=False)

### Crawler Example:

In [85]:
# Copy the url of the youtube video and paste it in the quotes
url = "https://www.youtube.com/watch?v=hrWd_-jDVB0&ab_channel=BBCEarthBBCEarthVerified"

In [91]:
#scrapedData
videoTitle

'Best Wild Animal Chases Part 2 | BBC Earth'

In [93]:
string = "ab1cd1ef"
string = string.replace("1", "") 

print(string)
# result: "abcdef"

abcdef


In [86]:
# Run this cell to create the csv file!
scrapedData, videoTitle = scrape_comments(url)
transform(scrapedData, videoTitle)

OSError: [Errno 22] Invalid argument: 'Best Wild Animal Chases Part 2 | BBC Earth.csv'

### Crawler: 

In [101]:
# Copy the url of the youtube video and paste it in the quotes
url = ""

In [102]:
# Run this cell to create the csv file!
scrapedData, videoTitle = scrape_comments(url)
transform(scrapedData, videoTitle)